In [2]:
import pandas as pd

In [39]:
df_weather=pd.read_csv('datasets/weather.csv',encoding='utf-8')

In [ ]:
from pprint import pprint
pprint(df_weather.columns.tolist())


In [41]:
df_val=pd.read_csv('datasets/weather_val.csv',encoding='utf-8')

In [ ]:
from pprint import pprint
pprint(df_val.columns.tolist())

In [4]:
df_weather.head()

,01-01 01:00,01-01 02:00,01-01 03:00,01-01 04:00,01-01 05:00,01-01 06:00,01-01 07:00,01-01 08:00,01-01 09:00,01-01 10:00,...,19-01 05:00.33,19-01 06:00.33,19-01 07:00.33,19-01 08:00.33,19-01 09:00.33,19-01 10:00.33,19-01 11:00.33,19-01 12:00.33,19-01 13:00.33,19-01 14:00.33
0,0.0,0.000009,0.000014,0.000014,0.000032,0.000149,0.000186,0.001388,0.001468,0.001468,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000017,0.000121,0.000152,0.001315,0.001386,0.001386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.000016,0.000103,0.000128,0.001226,0.001288,0.001288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.000015,0.000084,0.000103,0.001137,0.001190,0.001190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.000016,0.000070,0.000084,0.000832,0.000984,0.000984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_weather.shape

(7396, 24494)

In [6]:
coordinates = pd.read_csv('coordinates.csv',encoding='utf-8')

In [10]:
len(coordinates['longitude'])

7396

In [11]:
df_weather['longitude']=coordinates['longitude']

In [12]:
df_weather['latitude']=coordinates['latitude']

In [5]:
df_weather.to_csv('weather_with_coordinates.csv',index=False)

NameError: name 'df_weather' is not defined

In [3]:
df=pd.read_csv('datasets/weather_with_coordinates.csv',encoding='utf-8')

In [4]:
df.columns

Index(['01-01 01:00', '01-01 02:00', '01-01 03:00', '01-01 04:00',
       '01-01 05:00', '01-01 06:00', '01-01 07:00', '01-01 08:00',
       '01-01 09:00', '01-01 10:00',
       ...
       '19-01 07:00.33', '19-01 08:00.33', '19-01 09:00.33', '19-01 10:00.33',
       '19-01 11:00.33', '19-01 12:00.33', '19-01 13:00.33', '19-01 14:00.33',
       'longitude', 'latitude'],
      dtype='object', length=24496)

In [20]:
df.shape

(7396, 24496)

In [27]:
df.columns[23]

'01-01 24:00'

In [ ]:
from pprint import pprint
pprint(df.columns.tolist())

In [36]:
import pandas as pd

def reshape_rainfall_data(df: pd.DataFrame, start_year: int) -> pd.DataFrame:
    """
    Reshapes the rainfall data from a wide format to a long format,
    handling inconsistent time formats (including '24:00') and adding the correct year.

    Args:
        df: The original wide-format DataFrame.
        start_year: The first year of the 3-year data span.

    Returns:
        A long-format DataFrame with 'timestamp', 'latitude', 'longitude', 
        and 'rainfall' columns.
    """
    # 1. Isolate the identifier columns (location) and the value columns (time)
    id_vars = ['latitude', 'longitude']
    time_cols = [col for col in df.columns if col not in id_vars]

    # 2. Create a mapping from old column names to new, unambiguous timestamps
    new_column_names = {}
    current_year = start_year
    # We use (month, day) tuple to detect when the year rolls over
    last_month_day = (0, 0) 

    for col in time_cols:
        parts = col.split(' ')
        date_part = parts[0]
        time_part = parts[1]
        
        day, month = map(int, date_part.split('-'))

        # Logic to increment the year when the date rolls over
        # e.g., when we go from (12, 31) to (1, 1)
        if (month, day) < last_month_day:
            current_year += 1
        
        # Update the last seen date for the next iteration
        last_month_day = (month, day)

        # Construct the full, unambiguous timestamp string in a standard format
        # --- FIX: Use startswith to handle all '24:00' variations (e.g., '24:00.1') ---
        if time_part.startswith('24:00'):
            # Create a date object for the current day
            current_day_timestamp = pd.to_datetime(f"{current_year}-{month:02d}-{day:02d}")
            # Add one day to roll over to the next day at midnight
            next_day_timestamp = current_day_timestamp + pd.Timedelta(days=1)
            
            # Handle potential seconds attached to '24:00' (e.g., '24:00.1')
            time_part_corrected = time_part.replace('.', ':')
            time_components = time_part_corrected.split(':')
            
            final_timestamp = next_day_timestamp
            if len(time_components) > 2: # Check if seconds are present
                seconds_to_add = int(time_components[2])
                final_timestamp += pd.Timedelta(seconds=seconds_to_add)

            # Format as a string
            full_timestamp_str = final_timestamp.strftime("%Y-%m-%d %H:%M:%S")
        else:
            # NORMAL HANDLING for other times, including the '.33' seconds format
            time_part_corrected = time_part.replace('.', ':')
            
            # Ensure all timestamps have a seconds component for consistency
            if time_part_corrected.count(':') == 1:
                time_part_corrected += ':00'
            
            full_timestamp_str = f"{current_year}-{month:02d}-{day:02d} {time_part_corrected}"
        
        # Map the original column name to the new full timestamp
        new_column_names[col] = full_timestamp_str

    # 3. Rename the columns of the original DataFrame in one go
    df_renamed = df.rename(columns=new_column_names)

    # 4. Melt the DataFrame using the new, clean column names
    value_vars_renamed = list(new_column_names.values())
    df_long = pd.melt(df_renamed, id_vars=id_vars, value_vars=value_vars_renamed,
                      var_name='timestamp', value_name='rainfall')

    # 5. Convert the 'timestamp' column to a proper datetime object and set as index
    df_long['timestamp'] = pd.to_datetime(df_long['timestamp'])
    df_long.set_index('timestamp', inplace=True)
    
    # Sort the index to ensure chronological order
    df_long.sort_index(inplace=True)

    return df_long

# --- HOW TO USE THE CODE ---
# Assuming your original DataFrame is named 'df'
# You MUST specify the starting year of your data.
# For example, if your data starts in 2021:
STARTING_YEAR = 2021 

# Call the function to get the clean, long-format DataFrame
df_long_corrected = reshape_rainfall_data(df.copy(), STARTING_YEAR)

# Now, verify the output
print("--- Corrected df_long.head() ---")
print(df_long_corrected.head())
print("\n--- Corrected df_long.tail() ---")
print(df_long_corrected.tail())



--- Corrected df_long.head() ---
                     latitude  longitude  rainfall
timestamp                                         
2021-01-01 01:00:00      37.0       72.5       0.0
2021-01-01 01:00:00      37.0       72.6       0.0
2021-01-01 01:00:00      37.0       72.7       0.0
2021-01-01 01:00:00      37.0       72.8       0.0
2021-01-01 01:00:00      37.0       72.9       0.0

--- Corrected df_long.tail() ---
                     latitude  longitude  rainfall
timestamp                                         
2054-01-19 14:00:33      28.5       80.6       0.0
2054-01-19 14:00:33      28.5       80.7       0.0
2054-01-19 14:00:33      28.5       80.8       0.0
2054-01-19 14:00:33      28.5       80.9       0.0
2054-01-19 14:00:33      28.5       81.0       0.0


In [21]:
id_vars = ['latitude', 'longitude']
value_vars = [col for col in df.columns if col not in id_vars]

df_long = pd.melt(df, id_vars=id_vars, value_vars=value_vars, 
                  var_name='timestamp', value_name='rainfall')
df_long['timestamp'] = pd.to_datetime(df_long['timestamp'], format='%d-%m %H:%M', errors='coerce')

df_long.set_index('timestamp', inplace=True)

print(df_long.tail())

           latitude  longitude  rainfall
timestamp                               
NaT            28.5       80.6       0.0
NaT            28.5       80.7       0.0
NaT            28.5       80.8       0.0
NaT            28.5       80.9       0.0
NaT            28.5       81.0       0.0


In [18]:
print(df_long.shape)

(181157624, 3)


In [10]:
df_long.to_csv('datasets/rainfall.csv',index=False)

In [1]:
import pandas as pd

In [4]:
df=pd.read_csv('datasets/rainfall_data_transformed_final.csv',encoding='utf-8')

In [5]:
df.head()

,latitude,longitude,2010-06-01 00:00:00,2010-06-01 01:00:00,2010-06-01 02:00:00,2010-06-01 03:00:00,2010-06-01 04:00:00,2010-06-01 05:00:00,2010-06-01 06:00:00,2010-06-01 07:00:00,...,2024-09-30 14:00:00,2024-09-30 15:00:00,2024-09-30 16:00:00,2024-09-30 17:00:00,2024-09-30 18:00:00,2024-09-30 19:00:00,2024-09-30 20:00:00,2024-09-30 21:00:00,2024-09-30 22:00:00,2024-09-30 23:00:00
0,28.44,75.47,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28.44,75.72,0.0,0.0,0.0,0.0,0.0,0.0,7.823110e-07,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28.44,75.97,0.0,0.0,0.0,0.0,0.0,0.0,2.175569e-06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28.44,76.22,0.0,0.0,0.0,0.0,0.0,0.0,7.420778e-06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28.44,76.47,0.0,0.0,0.0,0.0,0.0,0.0,5.416572e-06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#transform this to a GeoPandas DataFrame and then I can perform spatial operations on it.